In [ ]:
import os
import pandas as pd
import numpy as np
import gensim
import string
import spacy
from tqdm import tqdm
from climdist.data import load as cd_load
import multiprocessing
import time
import json
import copy
import matplotlib.pyplot as plt
import string
from gensim.models.phrases import Phrases

In [ ]:
def sentences():
    with open('../data/processed/RZ_sentences.jsonl', 'r', encoding='utf8') as f:
        for line in tqdm(f.readlines()):
            sentence = json.loads(line)
            for key, value in sentence.items():
                yield value
                
def bigram_example(n):
    
    choices = [np.random.choice(256996) for i in range(n)]
    
    with open('../data/processed/RZ_sentences.jsonl', 'r', encoding='utf8') as f:
        for ix, line in tqdm(enumerate(f.readlines())):
            if ix in choices:
                print('\n\n', ix)
                sentence = json.loads(line)
                for key, value in sentence.items():
                    phrases = bigram_model[value]
                    for word in set([word for word in phrases if '_' in word]):
                        print(word)

In [ ]:
de_connector_words = ['von', 'vom', 'den', 'des', 'der', 'die', 'das', 'zu', 'ein',
                      'eine', 'einem', 'eines', 'nach', 'ohne', 'in']

In [ ]:
start = time.perf_counter()
bigram_model = Phrases(sentences(), min_count=50, threshold=10, connector_words=de_connector_words)
stop = time.perf_counter()
print(f'Finished in {round(stop-start, 2)} seconds')

In [ ]:
bigram_example(10)

In [ ]:
phrase_scores = phrase_model.find_phrases(sentences())

In [ ]:
[item for item in sorted(phrase_scores.items(), key=lambda x: x[1], reverse=True) if 'regen' in item[0]]

In [ ]:
class BigramIterator(): 
    def __iter__(self):
        for sentence in sentences():
            yield phrase_model[sentence]

In [ ]:
start = time.perf_counter()

w2v = gensim.models.Word2Vec(sentences=BigramIterator(),
                                    vector_size=100,
                                    min_count=10,
                                    workers=multiprocessing.cpu_count(),
                                    epochs=10)

print(f'Finished in {round(time.perf_counter()-start, 2)} seconds')